# Labelling training data
Given the relatively small size of our training data, we manually labelled the training data.

In [15]:
# Install imaging library Pillow
%pip install pillow

Note: you may need to restart the kernel to use updated packages.


In [16]:
import os
from PIL import Image
import xml.etree.ElementTree as ET

In [17]:
# Training data from Kaggle
unlabeled_data_folder = os.path.join("Data", "train")

# XML files created by LabelIMG
data_labels_folder = os.path.join("Labelled_Training_Data")

# Cropped images sorted by coin type
extracted_data_folder = os.path.join("Extracted_Training_Data")
os.makedirs(extracted_data_folder, exist_ok=True)

In [18]:
def crop_and_save_image(input_image_path, output_folder, object_list):
    # Open the input image
    with Image.open(input_image_path) as img:
        for index, object in enumerate(object_list):
            # Crop the image
            name = object['name']
            cropped_img = img.crop((int(object['xmin']), int(object['ymin']), int(object['xmax']), int(object['ymax'])))
            output_coin_type = os.path.join(output_folder, name)
            # Ensure the output folder exists
            os.makedirs(output_coin_type, exist_ok=True)
            
            # Construct the output image path, weird naming to avoid duplicates from same image, index placed before filetype
            image_name = os.path.basename(input_image_path[:-4]) + '_' + str(index) + '.JPG'
            output_image_path = os.path.join(output_coin_type, image_name)
            
            # Save the cropped image
            cropped_img.save(output_image_path)
            print(f'Cropped image saved to: {output_image_path}')

In [19]:
def read_xml_file_content(filepath):
    tree = ET.parse(filepath)
    root = tree.getroot()
    object_list = []
    for object in root.findall('object'):
        d = dict()
        d['name'] = object.findtext('name')
        bounding_box = object.find('bndbox')
        d['xmin'] = bounding_box.findtext('xmin')
        d['ymin'] = bounding_box.findtext('ymin')
        d['xmax'] = bounding_box.findtext('xmax')
        d['ymax'] = bounding_box.findtext('ymax')
        object_list.append(d)

    return object_list

In [20]:
for folder in os.listdir(unlabeled_data_folder):
    label_folder = os.path.join(data_labels_folder, folder)
    folder = os.path.join(unlabeled_data_folder, folder)
    for image in os.listdir(folder):
        # Find corresponding XML_file
        for label_file in os.listdir(label_folder):
            if (image[:-4] in label_file):
                label_filepath = os.path.join(label_folder, label_file)
                image_filepath = os.path.join(folder, image)
                object_list = read_xml_file_content(label_filepath)

                crop_and_save_image(image_filepath, extracted_data_folder, object_list)

Cropped image saved to: Extracted_Training_Data\5CHF\L1010277_0.JPG
Cropped image saved to: Extracted_Training_Data\2EUR\L1010277_1.JPG
Cropped image saved to: Extracted_Training_Data\0.2EUR\L1010277_2.JPG
Cropped image saved to: Extracted_Training_Data\0.2EUR\L1010277_3.JPG
Cropped image saved to: Extracted_Training_Data\0.5EUR\L1010277_4.JPG
Cropped image saved to: Extracted_Training_Data\0.5CHF\L1010279_0.JPG
Cropped image saved to: Extracted_Training_Data\0.2CHF\L1010279_1.JPG
Cropped image saved to: Extracted_Training_Data\2CHF\L1010279_2.JPG
Cropped image saved to: Extracted_Training_Data\0.1CHF\L1010279_3.JPG
Cropped image saved to: Extracted_Training_Data\0.2CHF\L1010281_0.JPG
Cropped image saved to: Extracted_Training_Data\0.05EUR\L1010281_1.JPG
Cropped image saved to: Extracted_Training_Data\0.05CHF\L1010281_2.JPG
Cropped image saved to: Extracted_Training_Data\0.1CHF\L1010281_3.JPG
Cropped image saved to: Extracted_Training_Data\0.1CHF\L1010281_4.JPG
Cropped image saved to: 

# Image segmentation

## Attempt with classical segmentation

## Object localization with transfer learning

# Classification

## Importing data

In [1]:
import os
import cv2 as cv

images_classes_path = "./Extracted_Training_data"
image_classes_list = os.listdir(images_classes_path)
num_classes = len(image_classes_list)
print("total {} classes".format(num_classes))

test_images_path = "output"
images_dict = {}
image_name_list = os.listdir(test_images_path)
for image_folder in image_name_list:
    images_dict[image_folder] = []
    images_path = os.path.join(test_images_path, image_folder)
    # Check if the path is a directory
    if not os.path.isdir(images_path):
        continue
    for image in os.listdir(images_path):
        path_to_coin_image = os.path.join(images_path, image)
        images_dict[image_folder].append(path_to_coin_image)
num_images = len(image_name_list)
print("total {} test images".format(num_images))

total 16 classes
total 0 test images


## Data augmentation

In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
import datetime
!rm -rf ./logs/

%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
import matplotlib.image as mpimg
import numpy as np


'rm' is not recognized as an internal or external command,
operable program or batch file.


### Import Required Libraries
This section imports the necessary libraries for image processing and augmentation `ImageDataGenerator`and related functions from `tensorflow.keras.preprocessing.image` are used for loading and augmenting images. The `os` module handles directory operation, and `tqdm` provides a progress bar for the augmentation process.

In [3]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.preprocessing import image
import os
from tqdm import tqdm
import random

### Initialize the ImageDataGenerator

An instance of `ImageDataGenerator`is created with specific augmentation parameters. The purpose of this data augmentation is to artificially expand the training dataset by generating new, altered versions of the existing coin images. This is particularly beneficial since our original training dataset is small, so augmentation will help to prevent overfitting and improve the model's ability to generalize to new data. 

For this task, the chosen augmentations are Rescaling, Rotation, Brightness Adjustment and Zoom. Rescaling of pixel values from [0,255] range to [0,1] is done to facilitate faster convergence during training. We apply a random rotation of up to 90 degrees as well, this is to make the model robust to changes in orientation, improving its ability to recognize coins regardless of rotation. Since lighting conditions vary slightly in the training data a brightness adjustment is also implemented. Finally a slight zoom in is implemented since the segmentation algorithm can cut of some edges of coins and therefore preparing the model for that in the training data makes generalize better to the test dataset.


In [4]:

# Create the directories to save the augmented images
augmented_data_path = 'path_to_save_augmented_images'
validation_data_path = 'path_to_save_validation_images'
os.makedirs(augmented_data_path, exist_ok=True)
os.makedirs(validation_data_path, exist_ok=True)

crazy_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=90,
    horizontal_flip=True,
    zoom_range=[0.8, 1.2],
    brightness_range=[0.5, 1.5],
    shear_range=0.2,
)


### Augment and Save Images
This loop iterates over each class directory and each image within that directory. Each image is loaded and converted to an array. The `datagen.flow` function generates augmented images which are saved to the specified directory.

In [33]:
import shutil
# Set the number of augmented images to generate per original image
num_augmented_images = 3

# Load the original training images
original_data_path = images_classes_path

# Calculate the total number of steps for the progress bar
all_files = sum([len(files) for r, d, files in os.walk(original_data_path)])
total_steps = int(all_files * 0.7 * num_augmented_images + all_files * 0.3)

# Initialize the progress bar
pbar = tqdm(total=total_steps, desc="Processing images")

# Split the original images into 70% for augmentation and 30% for validation
for class_dir in os.listdir(original_data_path):
    class_path = os.path.join(original_data_path, class_dir)
    augmented_class_path = os.path.join(augmented_data_path, class_dir)
    validation_class_path = os.path.join(validation_data_path, class_dir)
    os.makedirs(augmented_class_path, exist_ok=True)
    os.makedirs(validation_class_path, exist_ok=True)
    
    images = os.listdir(class_path)
    random.shuffle(images)
    split_index = int(len(images) * 0.7)
    
    images_to_augment = images[:split_index]
    images_for_validation = images[split_index:]
    
    # Save validation images to the validation data path without augmentation
    for img_name in images_for_validation:
        src_path = os.path.join(class_path, img_name)
        dst_path = os.path.join(validation_class_path, img_name)
        shutil.copy(src_path, dst_path)
        pbar.update(1)

    # Augment 70% of the images
    for img_name in images_to_augment:
        img_path = os.path.join(class_path, img_name)
        img = load_img(img_path)
        x = img_to_array(img)
        x = x.reshape((1,) + x.shape)
        
        # Generate and save augmented images
        i = 0
        for batch in crazy_datagen.flow(x, batch_size=1, save_to_dir=augmented_class_path, save_prefix='aug', save_format='jpeg'):
            i += 1
            pbar.update(1)
            if i >= num_augmented_images:
                break

pbar.close()

Processing images:  98%|█████████▊| 905/919 [00:58<00:00, 15.40it/s]


### Define Data generators for Training and validation
Two data generators are defined: one for training and one for validation. The images are rescaled, and the dataset is split into training and validation subsets. The images are resized to 224x224 picels, and a batch size of 32 is used. Different batch sizes were tested, but 32 was found to be a suitable batch size because of the small dataset.

In [34]:
# Define the data generators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255
)

val_datagen = ImageDataGenerator(
    rescale=1./255
)

train_generator = train_datagen.flow_from_directory(
    augmented_data_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    validation_data_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Test data generator remains the same
# test_datagen = ImageDataGenerator(rescale=1./255)
# test_generator = test_datagen.flow_from_directory(
#     test_images_path,
#     target_size=(224, 224),
#     shuffle=False,
#     batch_size=1,
#     class_mode='categorical'
# )

# Get filenames and number of samples
# filenames = test_generator.filenames
# nb_samples = len(filenames)

Found 2562 images belonging to 16 classes.
Found 274 images belonging to 16 classes.


### Validate Data Generators

In [35]:
# Print a batch for verification
x_batch, y_batch = next(train_generator)
print(f"Batch x_shape: {x_batch.shape}, Batch y_shape: {y_batch.shape}")

# Verify the data generators
train_batch = next(iter(train_generator))
val_batch = next(iter(val_generator))
print(train_batch[0].shape, train_batch[1].shape)
print(val_batch[0].shape, val_batch[1].shape)

print(f"Steps per epoch: {train_generator.samples // train_generator.batch_size}")
print(f"Validation steps: {val_generator.samples // val_generator.batch_size}")

Batch x_shape: (32, 224, 224, 3), Batch y_shape: (32, 16)
(32, 224, 224, 3) (32, 16)
(32, 224, 224, 3) (32, 16)
Steps per epoch: 80
Validation steps: 8


## Model configuration using Xception

In [7]:
import keras
from keras.layers import Input
from keras.models import Sequential, Model
from keras.optimizers import SGD, Adam

from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D, GlobalAveragePooling2D, BatchNormalization

### Loading and Customizing InceptionV3 Base Model
The pre-trained InceptionV3 model is loaded with weights from ImageNet, excluding the top fully connected layers. This allows us to customize the output layers for the coin classification task. 

In [8]:
from keras.applications.inception_v3 import InceptionV3
# from tensorflow.keras.applications import ResNet50
# from tensorflow.keras.applications import EfficientNetB0
# from keras.applications.xception import Xception
base_model = InceptionV3(include_top=False, # Since we will create our own
                    weights='imagenet', 
                    input_shape=(224, 224, 3))
# base_model.summary()

# base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# base_model.summary()

# base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# base_model.summary()

# base_model = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# base_model.summary()

### Building the Custom Output Layers

The ouput of the IncdeptionV3 base model is passed through a series of custom layers:

**Flatten:** Converst the feature maps to a 1D feature vector\
**BatchNormalization:** Normalizes the activations to improve training stability\
**Dense (512 units):** Fully connected layer with ReLy activation\
**Dropout (50%):** Regularization technique to prevent overfitting\
**Dense (num_classes):** Output layer with softmax actiavtion for the final coin classification

In [27]:
# x = inceptionv3_base.output
# # x = Flatten()(x)
# # x = BatchNormalization()(x)
# # x = Dense(512, activation='relu')(x)
# # x = Dropout(rate = .5)(x)
# x = GlobalAveragePooling2D()(x)
# x = Dense(1024, activation='relu')(x)
# x = Dropout(0.5)(x)
# x = Dense(512, activation='relu')(x)
# x = Dropout(0.3)(x)
# predictions = Dense(num_classes, activation='softmax')(x)

In [ ]:
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.inputs, outputs=predictions)
model.summary()

### Freezing of Base layers

This block freezes all the layer except the last 7 in order to retrain the pre-trained weights during initial training. The last 7 layers remain trainable to fine-tune them for coin classification. 

In [ ]:
# Freezing in order to only train the last 7 layers
for layer in model.layers[:]:
    layer.trainable = True
for layer in model.layers[:-7]:
    layer.trainable = False
for i, layer in enumerate(model.layers):
    print(i, layer.name, layer.trainable)
model.summary()


### Compiling the Model
The model is compiled with the Adam optimizer, which is chosen for its efficiency and adaptability. The learning rate is set to 0.0001. The loss function is categorical cross-entropy, which is suitibla for the multi-class coin classification. 

In [17]:
optimizer = Adam(learning_rate=0.0001)
# optimizer = SGD(learning_rate=0.0001, momentum=0.9)
model.compile(optimizer=optimizer,
              loss="categorical_crossentropy",
              metrics=["accuracy"])

### Setting Up Callbacks
Three callbacks are defined to enhance the training process:

**ModelCheckpoint:** Saves the model with the best validation accuracy\
**TensorBoard:** Logs training metrics for visualization.\
**EarlyStopping:** Stops training if the validation does not improve for 6 consecutive epochs, restoring the best weights.\

In [18]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import TensorBoard

checkpoint = ModelCheckpoint('model.keras',
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True,
                             mode='max')
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)


my_callbacks = [checkpoint, tensorboard_callback, early_stopping_callback]

### Training the Model

In [19]:
history = model.fit(train_generator,
          validation_data=val_generator,
          epochs=30,
          callbacks=my_callbacks)
print('Training done!')


Epoch 1/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 141s 5s/step - accuracy: 0.1519 - loss: 5.3837 - val_accuracy: 0.4885 - val_loss: 1.5666
Epoch 2/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 179s 7s/step - accuracy: 0.6017 - loss: 1.1679 - val_accuracy: 0.6107 - val_loss: 1.1468
Epoch 3/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 139s 5s/step - accuracy: 0.8202 - loss: 0.5376 - val_accuracy: 0.6336 - val_loss: 1.1122
Epoch 4/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 162s 6s/step - accuracy: 0.8908 - loss: 0.3408 - val_accuracy: 0.6336 - val_loss: 1.0378
Epoch 5/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 138s 5s/step - accuracy: 0.8930 - loss: 0.3145 - val_accuracy: 0.6870 - val_loss: 0.9315
Epoch 6/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 123s 5s/step - accuracy: 0.9524 - loss: 0.1667 - val_accuracy: 0.6870 - val_loss: 0.9934
Epoch 7/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 143s 5s/step - accuracy: 0.9678 - loss: 0.1257 - val_accuracy: 0.6870 - val_loss: 1.0160
Epoch 8/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 122s 4s/step - accuracy: 0.9855 - loss: 0.0824 - val_accuracy: 0.6947 - v

In [ ]:
model_label = ""
model.save(model_label)

In [ ]:
for layer in model.layers[:]:
    layer.trainable = True

In [ ]:
params = model.fit(train_generator, 
                                validation_data=val_generator, 
                                epochs=15,
                                callbacks=my_callbacks)

## Plot accuracies and losses

In [ ]:
def plot_history(history):
    acc = np.array(history.history['accuracy'])
    val_acc = np.array(history.history['val_accuracy'])
    loss = np.array(history.history['loss'])
    val_loss = np.array(history.history['val_loss'])

    epochs = range(len(acc))

    plt.figure(figsize=(12, 8))

    # Plot training and validation accuracy
    plt.subplot(2, 2, 1)
    plt.plot(epochs, acc, 'bo-', label='Training accuracy')
    plt.plot(epochs, val_acc, 'ro-', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plot training and validation loss
    plt.subplot(2, 2, 2)
    plt.plot(epochs, loss, 'bo-', label='Training loss')
    plt.plot(epochs, val_loss, 'ro-', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Plot accuracy
    plt.subplot(2, 2, 3)
    plt.plot(epochs, acc, 'bo-', label='Training accuracy')
    plt.plot(epochs, val_acc, 'ro-', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plot loss
    plt.subplot(2, 2, 4)
    plt.plot(epochs, loss, 'bo-', label='Training loss')
    plt.plot(epochs, val_loss, 'ro-', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Example usage:
# Assuming 'history' is the History object returned from model.fit
plot_history(history)
plot_history(params)

In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
import os
import time
import subprocess
from IPython.display import display, HTML

os.system('pkill -f "tensorboard"')
subprocess.Popen(["tensorboard", "--logdir", "logs/fit"])
time.sleep(5)
display(HTML(f'<a href="http://localhost:6006" target="_blank">Open TensorBoard</a>'))

## Use model on given test images


In [ ]:
def predict_images(path, verbose=False, show_image=False):
    # Load and display the image
    img = mpimg.imread(path)

    if show_image:
        plt.imshow(img)
        plt.axis('off')  # Hide axis
        plt.show()

    # Preprocess the image
    image = Image.open(path)
    image = image.convert('RGB')
    image = image.resize((224, 224))
    image = np.array(image) / 255.0  # Normalize the image
    image = np.expand_dims(image, axis=0)  # Expand dimensions to fit model input

    # Predict the probabilities
    probabilities = model.predict(image, verbose=0)

    # Get class labels
    class_labels = {v: k for k, v in val_generator.class_indices.items()}

    # Prepare data for a nice display
    sorted_indices = np.argsort(probabilities[0])[::-1]
    results = [(class_labels[i], probabilities[0][i]) for i in sorted_indices]

    # Print sorted probabilities and their corresponding class labels
    if verbose:
        print("Class".ljust(15), "Probability")
        print("-" * 30)
        for label, prob in results:
            print(f"{label.ljust(15)} : {prob:.4f}")

    # Return the class label with the highest probability
    return results[0][0]

In [ ]:
# For processing submission
import pandas as pd

# This will used as a template to build the submission.
if not os.path.exists("sample_submission.csv"):
    raise FileNotFoundError("sample_submission.csv not found!")
submission = pd.read_csv("sample_submission.csv")
submission.head()

# Perform prediction on all test images.
for key in images_dict:
    labels_count = {key: 0 for key in image_classes_list}
    for coin_path in images_dict[key]:
        predicted_class = predict_images(coin_path)
        labels_count[predicted_class] += 1

    row_to_write = submission.loc[submission["id"] == key[:-4]]

    for label in labels_count:
        row_to_write[label] = labels_count[label]

    submission.loc[submission["id"] == key[:-4]] = row_to_write


# Save the submission
submission.to_csv("submission.csv", index=False)

## Confusion matrix on validation set

In [ ]:
import pandas as pd
import seaborn as sns

Y_pred = model.predict(val_generator, 1167//32+1)
y_pred = np.argmax(Y_pred, axis=1)
cfm = confusion_matrix(val_generator.classes, y_pred)
cfm = np.around(cfm.astype('float')/cfm.sum(axis=1)[:, np.newaxis], decimals=2)
classes = ['0.1CHF', '0.1EUR', '0.01EUR', '0.2CHF', '0.2EUR', '0.02EUR', '0.5CHF', '0.05CHF', '0.5EUR', '0.05EUR', '1CHF', '1EUR', '2CHF', '2EUR', '5CHF', 'OOD']
cfm_pd = pd.DataFrame(cfm, index = classes, columns = classes)
figure = plt.figure(figsize=(8,8))
sns.heatmap(cfm_pd, annot=True, cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()
